# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import datetime
import os
#import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean

In [2]:
import matplotlib.pyplot as plt

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
data_path = './Project_data'
train_path = f'{data_path}/train'
val_path = f'{data_path}/val'


In [5]:
train_doc = np.random.permutation(open(f'{data_path}/train.csv').readlines())
val_doc = np.random.permutation(open(f'{data_path}/val.csv').readlines())
batch_size = 16 #experiment with the batch size

In [6]:
train_doc[0:10]

array(['WIN_20180926_16_54_08_Pro_Right_Swipe_new;Right_Swipe_new;1\n',
       'WIN_20180925_18_02_58_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n',
       'WIN_20180925_17_33_08_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_20180925_17_51_17_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_17_35_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_20180926_17_30_47_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_29_23_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180926_17_12_26_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_29_01_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180907_16_32_11_Pro_Stop Gesture_new;Stop Gesture_new;2\n'],
      dtype='<U88')

In [7]:
len(train_doc) // batch_size

41

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [8]:
def plot_images(images, captions=None, cmap=None ):
    n = len(images)
    rows = n // 4
    f, axes = plt.subplots(1, 4, sharey=True)
    f.set_figwidth(15)
    for ax,image in zip(axes, images):
        ax.imshow(image, cmap)

In [19]:
def generator(source_path, folder_list, batch_size):
    #print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = list(range(0, 30)) #create a list of image numbers you want to use for a particular video
    x = len(img_idx) # number of images for each video
    y = 120          # height of each input image
    z = 120          # width of each input image
    large=(360, 360, 3)
    small=(120, 160, 3)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #print("Reading ", source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #print("image shape is ", image.shape)
                    #plt.imshow(image)
                    #yield
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if (image.shape == large):
                        image = resize(image, (y, z), anti_aliasing=False)
                    else:
                        image = image[:, 20:140, :]
                    
                    #print("new image shape is ", image.shape)
                    #plt.imshow(image)
                    #yield
                    
                    #normalize 
                    image = image/255
                    
                    batch_data[folder,idx,:,:,0] = image[:, :, 0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:, :, 1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:, :, 2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        # write the code for the remaining data points which are left after full batches
        start_idx = num_batches * batch_size
        remaining_size = len(folder_list) - start_idx
        if remaining_size > 0:
            batch_data = np.zeros((remaining_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((remaining_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(start_idx, len(folder_list)): #iterate over the remaining folders
                imgs = os.listdir(source_path+'/'+ t[folder].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #print("Reading ", source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    image = io.imread(source_path+'/'+ t[folder].strip().split(';')[0]+'/'+imgs[item])
                    #print("image shape is ", image.shape)
                    #plt.imshow(image)
                    #yield
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if (image.shape == large):
                        image = resize(image, (y, z), anti_aliasing=False)
                    else:
                        image = image[:, 20:140, :]

                    #print("new image shape is ", image.shape)
                    #plt.imshow(image)
                    #yield

                    batch_data[folder - start_idx,idx,:,:,0] = image[:, :, 0] #normalise and feed in the image
                    batch_data[folder - start_idx,idx,:,:,1] = image[:, :, 1] #normalise and feed in the image
                    batch_data[folder - start_idx,idx,:,:,2] = image[:, :, 2] #normalise and feed in the image

            batch_labels[folder - start_idx, int(t[folder].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
    return None, None


In [10]:
# test generator 
#i = generator(data_path + "/train", train_doc, 32)

In [11]:
# data, labels = next(i)
# print(len(train_doc))
# print(len(data))
# print(len(labels))

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [20]:
curr_dt_time = datetime.datetime.now()
input_image_shape = (30, 120, 120, 3)
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)
num_classes = 5

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [21]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2
from keras import optimizers

#write your model here
model = Sequential()
model.add(Conv3D(32, (3, 3, 3), padding="valid",
                 input_shape=input_image_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(64, (3, 3, 3), padding="valid"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(16, kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [22]:
optimiser = optimizers.SGD() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 28, 118, 118, 32)  2624      
_________________________________________________________________
activation_7 (Activation)    (None, 28, 118, 118, 32)  0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 118, 118, 32)  128       
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 26, 116, 116, 32)  27680     
_________________________________________________________________
activation_8 (Activation)    (None, 26, 116, 116, 32)  0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 26, 116, 116, 32)  128       
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 13, 58, 58, 32)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [23]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [24]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [25]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [26]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


42/42 [==============================] - 133s 3s/step - loss: 9.8246 - categorical_accuracy: 0.2233 - val_loss: 13.1619 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2019-06-1618_03_07.198680/model-00001-9.94606-0.21870-13.16192-0.18000.h5
Epoch 2/10
42/42 [==============================] - 125s 3s/step - loss: 7.2807 - categorical_accuracy: 0.2118 - val_loss: 13.3459 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2019-06-1618_03_07.198680/model-00002-7.36815-0.21267-13.34591-0.20000.h5
Epoch 3/10
42/42 [==============================] - 124s 3s/step - loss: 5.5527 - categorical_accuracy: 0.1984 - val_loss: 8.2888 - val_categorical_accuracy: 0.1600

Epoch 00003: saving model to model_init_2019-06-1618_03_07.198680/model-00003-5.60644-0.19910-8.28882-0.16000.h5
Epoch 4/10
42/42 [==============================] - 124s 3s/step - loss: 5.2142 - categorical_accuracy: 0.2077 - val_loss: 4.2152 - val_categorical_accuracy: 0.1800

Epoc